In [1]:
import module.input_mutation_path as imp
import os

In [3]:
def keep_maximal_paths(paths):
    """最大パス（他に内包されないパス）のみ保持"""
    path_sets = [(set(path), path) for path in paths]
    maximal_paths = []
    
    for path_set, original_path in path_sets:
        is_maximal = True
        for other_set, _ in path_sets:
            if path_set != other_set and path_set.issubset(other_set):
                is_maximal = False
                break
        if is_maximal:
            maximal_paths.append(original_path)
    
    return maximal_paths

def save_paths_to_file(paths, filename):
    """パスをファイルに保存"""
    with open(filename, 'w') as f:
        for path in paths:
            f.write(','.join(path) + '\n')

In [5]:
def remove_subsets(paths):
    """内包されるパス（サブセット）を除去"""
    # まず完全一致の重複除去
    unique_paths = [list(item) for item in dict.fromkeys(tuple(path) for path in paths)]
    
    path_sets = [set(path) for path in unique_paths]
    filtered_paths = []
    
    for i, path_set in enumerate(path_sets):
        is_subset = False
        for j, other_set in enumerate(path_sets):
            if i != j and path_set.issubset(other_set) and path_set != other_set:
                is_subset = True  # 他のパスに完全に内包される
                break
        if not is_subset:
            filtered_paths.append(unique_paths[i])
    
    return filtered_paths

In [7]:
def generate_non_encompassing_sequences(paths):
    """
    非包括シーケンスを生成する関数
    互いに内包関係にない独立したパスのセットを作成
    
    Args:
        paths: 変異パスのリスト
        
    Returns:
        list: 非包括シーケンスのリスト
    """
    
    # Step 1: 完全一致の重複除去
    unique_paths = [list(item) for item in dict.fromkeys(tuple(path) for path in paths)]
    
    # Step 2: パスをセットに変換（比較用）
    path_data = [(set(path), path, len(path)) for path in unique_paths]
    
    # Step 3: 長さでソート（短いものから処理）
    path_data.sort(key=lambda x: x[2])
    
    non_encompassing = []
    
    for current_set, current_path, current_len in path_data:
        is_independent = True
        
        # 既に選択されたパスとの関係をチェック
        for selected_set, selected_path, selected_len in non_encompassing:
            # 内包関係をチェック（双方向）
            if (current_set.issubset(selected_set) or 
                selected_set.issubset(current_set)):
                is_independent = False
                break
        
        # 独立している場合のみ追加
        if is_independent:
            non_encompassing.append((current_set, current_path, current_len))
    
    # パスのみを返す
    return [path for _, path, _ in non_encompassing]

In [19]:
def extract_non_encompassing_sequences(paths):
    """
    互いに内包関係にない独立したシーケンスを抽出
    
    Args:
        paths: 変異パスのリスト
        
    Returns:
        list: 非内包シーケンスのリスト
    """
    
    # Step 1: 完全一致の重複除去
    unique_paths = [list(item) for item in dict.fromkeys(tuple(path) for path in paths)]
    
    # Step 2: 長さでソート（短いものから処理）
    unique_paths.sort(key=len)
    
    non_encompassing = []
    
    for current_path in unique_paths:
        current_set = set(current_path)
        is_independent = True
        
        # 既に選択されたパスとの内包関係をチェック
        for selected_path in non_encompassing:
            selected_set = set(selected_path)
            
            # 双方向の内包関係をチェック
            if (current_set.issubset(selected_set) or 
                selected_set.issubset(current_set)):
                is_independent = False
                break
        
        # 独立している場合のみ追加
        if is_independent:
            non_encompassing.append(current_path)
    
    return non_encompassing

def extract_maximal_independent_sequences(paths):
    """
    最大かつ独立したシーケンスを抽出（長いパス優先）
    
    Args:
        paths: 変異パスのリスト
        
    Returns:
        list: 最大独立シーケンスのリスト
    """
    
    # 完全一致の重複除去
    unique_paths = [list(item) for item in dict.fromkeys(tuple(path) for path in paths)]
    
    # 長さで降順ソート（長いものから処理）
    unique_paths.sort(key=len, reverse=True)
    
    maximal_independent = []
    
    for current_path in unique_paths:
        current_set = set(current_path)
        is_independent = True
        
        # 既に選択されたパスとの関係をチェック
        for selected_path in maximal_independent:
            selected_set = set(selected_path)
            
            # 内包関係をチェック
            if (current_set.issubset(selected_set) or 
                selected_set.issubset(current_set)):
                is_independent = False
                break
        
        if is_independent:
            maximal_independent.append(current_path)
    
    return maximal_independent

In [ ]:
def is_sublist(sublist, mainlist):
    n = len(mainlist)
    m = len(sublist)
    if m == 0:  # 空のリストは常に任意のリストの部分リストとみなす
        return True
    if m > n:   # 部分リストがメインリストより長い場合は、部分リストになりえない
        return False
    
    # メインリストの各開始位置から部分リストと一致するかチェック
    for i in range(n - m + 1):
        if mainlist[i : i + m] == sublist:
            return True
    return False

def get_non_encompassed_sequences_2d(sequences_2d):
    if not sequences_2d:
        return []

    # set() を使うために、内部リストをタプルに変換してハッシュ可能にする
    # 重複を除去し、長さに応じてソート
    unique_sequences = [list(item) for item in dict.fromkeys(tuple(path) for path in sequences_2d)]
    
    # 長さでソート（短いものが先に来るように）
    sorted_sequences = sorted(unique_sequences, key=len)

    non_encompassed = []

    for i, current_seq in enumerate(sorted_sequences):
        is_encompassed = False
        
        # 現在のシーケンスが他のどのシーケンスに内包されていないかを確認
        for j, other_seq in enumerate(sorted_sequences):
            if i == j:  # 自分自身との比較はスキップ
                continue

            # current_seq が other_seq に完全に内包されているか
            # is_sublist ヘルパー関数を使用
            if is_sublist(current_seq, other_seq):
                is_encompassed = True
                break # 一つでも内包されていれば、それは非内包ではないので次のシーケンスへ

        if not is_encompassed:
            non_encompassed.append(current_seq)

    return non_encompassed

In [26]:
test_list = [
    ['A', 'B', 'C'],
    ['A', 'B', 'C'],
    ['A', 'B'],
    ['A', 'B', 'C', 'D'],
    ['A', 'B', 'C', 'D', 'E'],
    ['A', 'B', 'C', 'F'],
    ['A', 'B', 'C', 'D', 'E', 'F'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
    ['A', 'B', 'C', 'D', 'E', 'H'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'H'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
    ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
    ]

print(keep_maximal_paths(test_list))
print(remove_subsets(test_list))
print(generate_non_encompassing_sequences(test_list))
print(extract_non_encompassing_sequences(test_list))
print(extract_maximal_independent_sequences(test_list))
print(get_non_encompassed_sequences_2d(test_list))

[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']]
[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']]
[['A', 'B']]
[['A', 'B']]
[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']]
[['A', 'B', 'C', 'F'], ['A', 'B', 'C', 'D', 'E', 'H'], ['A', 'B', 'C', 'D', 'E', 'F', 'H'], ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']]


In [18]:
encom_seq = []
for tl in test_list:
    flag = 0
    for es in encom_seq:
        if set(tl).issubset(set(es)):
            flag = 1
            break
    if flag == 0:
        encom_seq.append(tl)  
print(encom_seq)

[['A', 'B', 'C'], ['A', 'B', 'C', 'D'], ['A', 'B', 'C', 'D', 'E'], ['A', 'B', 'C', 'F'], ['A', 'B', 'C', 'D', 'E', 'F'], ['A', 'B', 'C', 'D', 'E', 'F', 'G'], ['A', 'B', 'C', 'D', 'E', 'H'], ['A', 'B', 'C', 'D', 'E', 'F', 'H'], ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']]


In [15]:
test_list[0] in test_list[1]

False

In [4]:

# データセット設定
dataset_config = {
    'strains': ['B.1.1.7','P.1','BA.2','BA.1.1','BA.1','B.1.617.2','B.1.351','B.1.1.529'],
    'usher_dir': '../usher_output/',
}

data_config = {
    'nmax': 100000000,
    'nmax_per_strain': 1000000
}


for strain in dataset_config['strains']:
    names, lengths, base_HGVS_paths = imp.input(
        [strain], 
        dataset_config['usher_dir'], 
        nmax=data_config['nmax'], 
        nmax_per_strain=data_config['nmax_per_strain']
    )
    set_list1 = [list(item) for item in dict.fromkeys(tuple(path) for path in base_HGVS_paths)]
    #set_list2 = keep_maximal_paths(set_list1)
    print(f"\n{strain} のデータ:")
    print(f"重複除去後の長さ: {len(set_list1)}")
    #print(f"最大パス保持後の長さ: {len(set_list2)}")

names, lengths, base_HGVS_paths = imp.input(
        dataset_config['strains'],
        dataset_config['usher_dir'], 
        nmax=data_config['nmax'], 
        nmax_per_strain=data_config['nmax_per_strain']
)
print(f"全体のデータ:")
print(f"データ数: {len(base_HGVS_paths)}")
set_list1 = [list(item) for item in dict.fromkeys(tuple(path) for path in base_HGVS_paths)]
print(f"重複除去後の長さ: {len(set_list1)}")

[INFO] import: ../usher_output/B.1.1.7/0/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/1/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/2/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/3/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/4/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/5/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/6/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/7/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/8/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/9/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/10/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/11/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/12/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/13/mutation_paths_B.1.1.7.tsv
[INFO] import: ../usher_output/B.1.1.7/14/mu

In [ ]:
set_list2 = keep_maximal_paths(set_list1)
print(f"最大パス保持後の長さ: {len(set_list2)}")

In [ ]:
file_path1 = os.path.join(dataset_config['usher_dir'], 'unique_mutaion_paths.tsv')
file_path2 = os.path.join(dataset_config['usher_dir'], 'non_encompassed_mutaion_paths.tsv')

save_paths_to_file(set_list1, file_path1)
save_paths_to_file(set_list2, file_path2)

In [7]:
for i in range(0,10):
    print("a",i)
    for i in range(10,0,-1):
        print(" b",i)

a 0
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 1
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 2
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 3
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 4
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 5
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 6
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 7
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 8
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
a 9
 b 10
 b 9
 b 8
 b 7
 b 6
 b 5
 b 4
 b 3
 b 2
 b 1
